In [3]:
import tushare as ts
import pandas as pd
from pandas import Series, DataFrame

# pro = ts.pro_api("6848a1962cda9db1a4f7410fc876acaced030e98e635923e2d2fad66")
# df = pro.daily(**{
#   'ts_code': '601857.SH'
# })
df = pd.read_csv("../../../daily/601258.SH.csv")
df


,Unnamed: 0,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,0,601258.SH,20220805,1.29,1.30,1.28,1.30,1.29,0.01,0.7752,394283.40,50787.285
1,1,601258.SH,20220804,1.29,1.30,1.27,1.29,1.29,0.00,0.0000,494761.00,63506.888
2,2,601258.SH,20220803,1.28,1.32,1.28,1.29,1.28,0.01,0.7813,638107.00,82817.613
3,3,601258.SH,20220802,1.34,1.34,1.27,1.28,1.34,-0.06,-4.4776,1150092.61,149213.134
4,4,601258.SH,20220801,1.33,1.35,1.32,1.34,1.34,0.00,0.0000,625285.49,83589.981
...,...,...,...,...,...,...,...,...,...,...,...,...
2706,2706,601258.SH,20110505,34.24,34.85,33.70,34.40,34.71,-0.31,-0.8900,24366.18,83478.689
2707,2707,601258.SH,20110504,33.18,35.85,33.01,34.71,33.61,1.10,3.2700,76175.56,264329.646
2708,2708,601258.SH,20110503,34.01,34.01,32.81,33.61,34.38,-0.77,-2.2400,58079.33,193995.536
2709,2709,601258.SH,20110429,34.10,35.18,33.34,34.38,34.58,-0.20,-0.5800,65108.62,222856.479


In [ ]:
df.drop(labels='Unnamed: 0', axis='columns', inplace=True)
df['trade_date'] = pd.to_datetime(df['trade_date'], format='%Y%m%d')
df.set_index('trade_date', inplace=True)
df = df.loc['2022':]
df.sort_index(inplace=True)

/var/folders/4l/5d0kf6yx68n51tswgt17s0y40000gn/T/ipykernel_13044/616814872.py:4: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df = df.loc['2022':]


In [ ]:
df

,ts_code,open,high,low,close,pre_close,change,pct_chg,vol,amount
trade_date,,,,,,,,,,
2022-01-04,601258.SH,1.64,1.67,1.63,1.66,1.64,0.02,1.2195,1021739.13,168650.492
2022-01-05,601258.SH,1.66,1.68,1.64,1.65,1.66,-0.01,-0.6024,832557.54,137841.389
2022-01-06,601258.SH,1.65,1.67,1.64,1.66,1.65,0.01,0.6061,825867.00,136739.497
2022-01-07,601258.SH,1.66,1.66,1.63,1.64,1.66,-0.02,-1.2048,1148696.00,189082.936
2022-01-10,601258.SH,1.63,1.64,1.62,1.63,1.64,-0.01,-0.6098,654130.19,106611.512
...,...,...,...,...,...,...,...,...,...,...
2022-08-01,601258.SH,1.33,1.35,1.32,1.34,1.34,0.00,0.0000,625285.49,83589.981
2022-08-02,601258.SH,1.34,1.34,1.27,1.28,1.34,-0.06,-4.4776,1150092.61,149213.134
2022-08-03,601258.SH,1.28,1.32,1.28,1.29,1.28,0.01,0.7813,638107.00,82817.613


In [ ]:
long = 30
ma5 = df['close'].rolling(5).mean()[long:]
ma21 = df['close'].rolling(long).mean()[long:]

s1 = ma5 > ma21
s2 = ma5 < ma21

golden_ex = s1 & s2.shift(1)
death_ex = s1.shift(1) & s2
df = df[long:]

golden_date = df[golden_ex].index
death_date = df[death_ex].index

golden_s = Series(data=1, index=golden_date)
death_s = Series(data=0, index=death_date)
s = pd.concat(objs=(golden_s, death_s))
s.sort_index(inplace=True)

money = 5000
hold = 0

for i in range(0, len(s)):
  date = s.index[i]
  price = df.loc[date]['open']
  if s[i] == 1:
    hand_buy = money // (price * 100)
    if hand_buy > 0:
      hold_buy = hand_buy * 100
      hold += hold_buy
      cost = hold_buy * price
      money -= cost
      print(date.strftime(format='%Y-%m-%d'), ' 买入', hold, '股, 价格: ', round(price, 2), ', 花费: ', round(cost,2), ', 剩余资金: ', round(money,2))
  else:
    if hold > 0:
      hold_sell = hold
      amount_sell = hold_sell * price
      money += amount_sell
      hold = 0
      print(date.strftime(format='%Y-%m-%d'), ' 卖出', hold_sell, ', 价格: ', round(price, 2), ', 收入: ', round(amount_sell,2), ', 盈亏: ', round(money,2))

2022-03-07  买入 3200.0 股, 价格:  1.56 , 花费:  4992.0 , 剩余资金:  8.0
2022-03-09  卖出 3200.0 , 价格:  1.51 , 收入:  4832.0 , 盈亏:  4840.0
2022-05-23  买入 3500.0 股, 价格:  1.36 , 花费:  4760.0 , 剩余资金:  80.0
2022-07-05  卖出 3500.0 , 价格:  1.37 , 收入:  4795.0 , 盈亏:  4875.0
